<a href="https://colab.research.google.com/github/joseauto/Programa-o_Web_-CGI-_02/blob/main/Programa%C3%A7%C3%A3o_Web_(CGI)_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aluno:
* José Auto Araujo Neto

# Configuração do Servidor

* No arquivo de configuração do Apache

In [ ]:
<Directory "/caminho/para/cgi-bin">
    Options +ExecCGI
    AddHandler cgi-script .cgi
</Directory>

* Certifique-se de que o CGI esteja habilitado no Apache

In [ ]:
sudo a2enmod cgi
sudo systemctl restart apache2

# Página HTML

* Crie o arquivo calculadora.html que será a interface do usuário para inserir números e escolher uma operação.

In [ ]:
<!DOCTYPE html>
<html lang="pt-BR">
<head>
    <meta charset="UTF-8">
    <title>Calculadora</title>
</head>
<body>
    <h1>Calculadora Simples</h1>
    <form action="/cgi-bin/calculadora.cgi" method="post">
        <label for="num1">Número 1:</label>
        <input type="text" id="num1" name="num1" required><br><br>

        <label for="num2">Número 2:</label>
        <input type="text" id="num2" name="num2" required><br><br>

        <label for="operacao">Operação:</label>
        <select id="operacao" name="operacao">
            <option value="add">Adição</option>
            <option value="sub">Subtração</option>
            <option value="mul">Multiplicação</option>
            <option value="div">Divisão</option>
        </select><br><br>

        <button type="submit">Calcular</button>
    </form>
</body>
</html>

# Script Bash CGI (Calculadora)

* O script calculadora.cgi processa os dados enviados pelo formulário via POST, executa a operação e exibe o resultado. Coloque esse script no diretório cgi-bin configurado no Apache.

* Crie o arquivo calculadora.cgi com o seguinte conteúdo:

In [ ]:
#!/bin/bash

echo "Content-Type: text/html"
echo ""

read_input() {
    local input
    read -r input
    echo "$input"
}

DATA=$(read_input)

NUM1=$(echo "$DATA" | grep -oP 'num1=\K[0-9]+(\.[0-9]+)?')
NUM2=$(echo "$DATA" | grep -oP 'num2=\K[0-9]+(\.[0-9]+)?')
OPERACAO=$(echo "$DATA" | grep -oP 'operacao=\K[^&]+')

if [[ -z "$NUM1" || -z "$NUM2" ]]; then
    echo "<html><body><h2>Erro: Ambos os campos devem conter valores numéricos válidos.</h2></body></html>"
    exit 1
fi

case "$OPERACAO" in
    add)
        RESULTADO=$(echo "$NUM1 + $NUM2" | bc)
        OPERACAO_SIMBOLO="+"
        ;;
    sub)
        RESULTADO=$(echo "$NUM1 - $NUM2" | bc)
        OPERACAO_SIMBOLO="-"
        ;;
    mul)
        RESULTADO=$(echo "$NUM1 * $NUM2" | bc)
        OPERACAO_SIMBOLO="*"
        ;;
    div)
        if [[ "$NUM2" == "0" ]]; then
            echo "<html><body><h2>Erro: Divisão por zero não é permitida.</h2></body></html>"
            exit 1
        fi
        RESULTADO=$(echo "scale=2; $NUM1 / $NUM2" | bc)
        OPERACAO_SIMBOLO="/"
        ;;
    *)
        echo "<html><body><h2>Erro: Operação inválida.</h2></body></html>"
        exit 1
        ;;
esac

echo "<html><body>"
echo "<h1>Resultado da Operação</h1>"
echo "<p>${NUM1} ${OPERACAO_SIMBOLO} ${NUM2} = ${RESULTADO}</p>"
echo "</body></html>"

# Testando Aplicação

 * Acesse a página calculadora.html no navegador (exemplo, http://localhost/calculadora.html).




* Preencha os campos e selecione uma operação.

* Observação: Certifique-se de que o script calculadora.cgi tem permissão de execução:

In [ ]:
chmod +x /caminho/para/cgi-bin/calculadora.cgi